## Working with data

In [2]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

/Users/krit/miniconda3/envs/mlenv/lib/python3.12/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/krit/miniconda3/envs/mlenv/lib/python3.12/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <367D4265-B20F-34BD-94EB-4F3EE47C385B> /Users/krit/miniconda3/envs/mlenv/lib/python3.12/site-packages/torchvision/image.so
  Reason: tried: '/Users/krit/miniconda3/envs/mlenv/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/krit/miniconda3/envs/mlenv/lib/python3.12/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/krit/miniconda3/envs/mlenv/lib/python3.12/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/krit/miniconda3/envs/mlenv/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wro

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data", # Folder
    train=True, # Train if False test
    download=True, # Download to local
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
batch_size = 64 # No. of sample in each batch.

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    # batch N, channels C, depth D, height H, width W
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models

In [ ]:
from torch import nn

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.300571  [   64/60000]
loss: 2.292586  [ 6464/60000]
loss: 2.273026  [12864/60000]
loss: 2.266456  [19264/60000]
loss: 2.257135  [25664/60000]
loss: 2.220899  [32064/60000]
loss: 2.234702  [38464/60000]
loss: 2.197364  [44864/60000]
loss: 2.194612  [51264/60000]
loss: 2.172555  [57664/60000]
Test Error: 
 Accuracy: 44.1%, Avg loss: 2.165763 

Epoch 2
-------------------------------
loss: 2.170415  [   64/60000]
loss: 2.168338  [ 6464/60000]
loss: 2.118921  [12864/60000]
loss: 2.131489  [19264/60000]
loss: 2.097644  [25664/60000]
loss: 2.026586  [32064/60000]
loss: 2.056881  [38464/60000]
loss: 1.981495  [44864/60000]
loss: 1.980520  [51264/60000]
loss: 1.919438  [57664/60000]
Test Error: 
 Accuracy: 58.2%, Avg loss: 1.921637 

Epoch 3
-------------------------------
loss: 1.945169  [   64/60000]
loss: 1.928097  [ 6464/60000]
loss: 1.824669  [12864/60000]
loss: 1.850027  [19264/60000]
loss: 1.765560  [25664/60000]
loss: 1.699257  [32064/600

## Saving Models

In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Loading Models

In [10]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [11]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
